# Project Outline

This project looks at the stock price of Roku. By doing a time series analysis and also a sentiment analysis on social media for the stock price of Roku we will try and predict future prices.

Project flow:
 * Gather stock price
 * Gather posts from twitter and reddit
 * Do a sentiment analysis for each day
 * Do a time series analysis on both stock price and sentiment individually
 * Combine the sentiment and stock price to try and predict future prices

# Packages used

In [1]:
# Installing important packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import twint
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
from psaw import PushshiftAPI
import praw
import datetime as dt
import yfinance as yf
import requests 
import requests.auth
from Config import *
import time

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)


# This is so twint will be able to work
import nest_asyncio
nest_asyncio.apply()

# Gathering stock data

In [2]:
# Downloading Roku data from yfinance
stock_data = yf.download('ROKU','2017-09-28','2021-05-31')

[*********************100%***********************]  1 of 1 completed


In [3]:
# Turning the stock data index into a list we can itterate through
stock_days = stock_data.index

# Making list to itterate through for Reddit Data
stock_days_central = stock_days.tz_localize(tz='US/Central')
stock_days_itter = [int(x.timestamp()) for x in stock_days_central]

# Making list to itterate through for twitter data
twitter_stock_days = stock_data.index.strftime('%Y-%m-%d')

In [4]:
# Checking that the time is correct
local_datetime_converted = dt.datetime.fromtimestamp(stock_days_itter[0])
print(local_datetime_converted)

2017-09-28 00:00:00


# Gathering social media data

## Twitter Data

### Twitter Data Test

#### CSV route

In [ ]:
# # Gathering twitter data

# # Configure
# config = twint.Config()
# config.Search = "$ROKU"
# config.Since = '2021-05-09'
# config.Until = '2021-05-11'
# config.Lang = 'en'
# config.Store_csv = True
# config.Min_likes = 5
# config.Limit = 20
# config.Popular_tweets = True
# config.Output = '/Users/stevenkyle/Documents/Flatiron/Capstone/Capstone'

# # Running query search
# twint.run.Search(config)


In [ ]:
# test = pd.read_csv('tweets.csv')
# test.dropna(axis=1, inplace=True)
# test.drop_duplicates(inplace=True)
# test

In [ ]:
# test.columns

#### Pandas route

In [25]:
# Configure
c = twint.Config()
c.Search = '$ROKU'
c.Since = '2021-05-05'
c.Until = '2021-05-07'
c.Lang = 'en'
c.Limit = 20  #Has to be increments of 20
c.Pandas = True
c.Pandas_clean = True
c.Pandas_au = True
c.Popular_tweets = True
c.Min_likes = 10

# Running the search
twint.run.Search(c)

# Storing to dataframe
Tweets_df = twint.storage.panda.Tweets_df

# Dropping unwanted columns
Tweets_df.drop(columns=['conversation_id','created_at','place','hashtags','user_id',
                       'user_id_str','name','link','urls','photos','video','thumbnail','retweet',
                       'quote_url','near','geo','source','user_rt_id','user_rt','retweet_id','reply_to',
                       'retweet_date','translate','trans_src','trans_dest'], inplace=True)

1390048334119833601 2021-05-05 15:58:43 -0500 <ParrotStock> Whelp, Twitter is cooperating about as well as my brokerage today 😂   Going to try and post this for the 3rd time today...  No moves and trying to ignore the ST noise 😊  Despite the effort, the reds prevailed $PTON $ROKU $ATER $MGNI $IDN $SHOP $CRWD $DMTK $SKLZ  Another day 🦜  https://t.co/0lOr4sHw1l
1390047541891264516 2021-05-05 15:55:34 -0500 <StockDweebs> Day 2: Trying to get to 6/7 - figures  Starting Balance: $50,000 Unrealized P&amp;L: -$431.18 Realized P&amp;L: $0.00  Current Positions: $ROKU, $NVDA Trade Ideas: $TWTR, $BABA, $NFLX, $SQ, $SAVE, $ATVI, $SHAK  Comments, portfolio, and spreadsheets screenshots attached.  https://t.co/XMhEjSPMAc
1390029715566649346 2021-05-05 14:44:44 -0500 <StockMKTNewz> Top 10 holdings of Cathie Wood &amp; Ark Invest's $ARKK Ark Innovation ETF as of today 5/5  1 Tesla $TSLA 2 Teladoc $TDOC 3 $ROKU 4 Square $SQ 5 Shopify $SHOP 6 Zillow $Z $ZG 7 Zoom $ZM 8 Spotify $SPOT 9 Twilio $TWLO 10 E

In [26]:
# Checking dataframe
Tweets_df

,id,date,timezone,tweet,language,cashtags,username,day,hour,nlikes,nreplies,nretweets,search
0,1390048334119833601,2021-05-05 15:58:43,-0500,"Whelp, Twitter is cooperating about as well as...",en,"[pton, roku, ater, mgni, idn, shop, crwd, dmtk...",ParrotStock,3,15,60,25,2,$ROKU
1,1390047541891264516,2021-05-05 15:55:34,-0500,Day 2: Trying to get to 6/7 - figures Startin...,en,"[roku, nvda, twtr, baba, nflx, sq, save, atvi,...",StockDweebs,3,15,253,33,6,$ROKU
2,1390029715566649346,2021-05-05 14:44:44,-0500,Top 10 holdings of Cathie Wood &amp; Ark Inves...,en,"[arkk, tsla, tdoc, roku, sq, shop, z, zg, zm, ...",StockMKTNewz,3,14,45,6,3,$ROKU
3,1390019124571500544,2021-05-05 14:02:39,-0500,$roku down 20% in 4 days on notbing,en,[roku],Crussian17,3,14,86,29,1,$ROKU
4,1390008384133337090,2021-05-05 13:19:58,-0500,If $roku drops tmrw on earnings im doubling do...,en,[roku],Crussian17,3,13,57,12,1,$ROKU
5,1389991777151569924,2021-05-05 12:13:59,-0500,Bought $ROKU 15OCT21 350.00c small. This is m...,en,[roku],data168,3,12,37,4,0,$ROKU
6,1389940123953991685,2021-05-05 08:48:44,-0500,@phoenixvalue $ROKU,und,[roku],WallSt_Dropout,3,08,11,1,0,$ROKU
7,1389939852368560132,2021-05-05 08:47:39,-0500,"Right now, $CMCSA is valued at 6X $ROKU... No...",en,"[cmcsa, roku, roku]",anandchokkavelu,3,08,38,4,1,$ROKU
8,1389927011745996804,2021-05-05 07:56:37,-0500,GM Stratters! Up and down we go. I prefer the ...,en,"[fdx, ma, zm, team, roku, pypl, nvda, okta, etsy]",r3dpepsi,3,07,24,4,7,$ROKU
9,1389913397026770944,2021-05-05 07:02:31,-0500,"Yellen's Error, 'Inflation Scare', Cyclical Se...",en,"[aapl, roku, snap, dash, rtx, lmt, amat, amd, ...",Sarge986,3,07,17,2,4,$ROKU


In [27]:
Tweets_df.columns

Index(['id', 'date', 'timezone', 'tweet', 'language', 'cashtags', 'username',
       'day', 'hour', 'nlikes', 'nreplies', 'nretweets', 'search'],
      dtype='object')

In [33]:
Tweets_df.iloc[1]['tweet']

'Day 2: Trying to get to 6/7 - figures  Starting Balance: $50,000 Unrealized P&amp;L: -$431.18 Realized P&amp;L: $0.00  Current Positions: $ROKU, $NVDA Trade Ideas: $TWTR, $BABA, $NFLX, $SQ, $SAVE, $ATVI, $SHAK  Comments, portfolio, and spreadsheets screenshots attached.  https://t.co/XMhEjSPMAc'

### Looping through and collecting Twitter data

In [7]:
for i in range(len(twitter_stock_days)):
   
    # Configure
    c = twint.Config()
    c.Search = '$ROKU'
    
    ### Change since and until
    if i < 922:
        c.Since = twitter_stock_days[i]
        next_day = i+1
        c.Until = twitter_stock_days[next_day]
    
    elif i == 922:
        c.Since = twitter_stock_days[i]
        c.Until = '2021-05-31'
        
    c.Lang = 'en'
    c.Limit = 100  #Has to be increments of 20
    c.Pandas = True
    c.Pandas_clean = True
    c.Pandas_au = True
    c.Popular_tweets = True
    c.Min_likes = 10

    # Running the search
    twint.run.Search(c)

    # Storing to dataframe
    Tweets_df = twint.storage.panda.Tweets_df
    
    if len(Tweets_df) > 0:
        # Dropping unwanted columns
        Tweets_df.drop(columns=['conversation_id','created_at','place','hashtags','user_id',
                               'user_id_str','name','link','urls','photos','video','thumbnail','retweet',
                               'quote_url','near','geo','source','user_rt_id','user_rt','retweet_id','reply_to',
                               'retweet_date','translate','trans_src','trans_dest'], inplace=True)
    
        Tweets_df.sort_values(by='nlikes', axis=0, ascending=False, inplace=True)
    
        # Take the top 5 in dataframe to write
        df_to_write = Tweets_df[0:5]
    
        # Writting to a csv
        if i == 0:
            df_to_write.to_csv(path_or_buf='/Users/stevenkyle/Documents/Flatiron/Capstone/Capstone/TwitterData.csv',index=False)
        elif i > 0:
            df_to_write.to_csv('/Users/stevenkyle/Documents/Flatiron/Capstone/Capstone/TwitterData.csv', mode='a', header=False, index=False)
    
    time.sleep(3)

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
914147418316398592 2017-09-30 10:18:23 -0500 <MadMoneyOnCNBC> Mystified by $ROKU's IPO? @JimCramer's breaking it down  https://t.co/zxbDI4CVcg  https://t.co/OvjfOLjNcc
913892627845570560 2017-09-29 17:25:56 -0500 <SmashTheBid> $ROKU Long. One Ticker, a few trades. Couldve done a lot better but I'll take it.  https://t.co/rA3MVCi7c8
913892249481633792 2017-09-29 17:24:26 -0500 <SmashTheBid> $ROKU final chart.  https://t.co/yB5SKG3EqN
913843670872444928 2017-09-29 14:11:24 -0500 <EbogeyEd> Love #Pennystocks Nice Friday for my 2 accounts over $1200 $VCEL $IZEA $GLUU $ROKU #profits #FinancialFreedom TY @tbohen @timothysykes @thehonestcroock @kroyrunner89 All helped me to realize the potential.  https://t.co/RbbP7YRXdf
913841066473869313 2017-09-29 14:01:03 -0500 <Stocktwits> $ROKU is already up almost 100% since its IPO. Here's why they went public and what they do  https://t.co/VwIC9Hz7VL
913765506456473600 2

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
939477117674426369 2017-12-09 06:49:34 -0500 <ZorTrades> 10. recent examples $ROKU $CDNA $QURE 11. Stocks down multiple days in a row is a good scan, many stocks that are up the most in the short-term have this one set up; down 3days or more up today.  https://t.co/WtpOlQeBb0
939286730183606273 2017-12-08 18:13:02 -0500 <traderstewie> As promised!!!  Here's the $ROKU Step-by-Step Trade Analysis !!!    https://t.co/kIGLcrCA2S   Took me 6 hours to write this post, I'm sure many of you will find it helpful !!  Let's kill it again NEXT WEEK!
939241563565187072 2017-12-08 15:13:33 -0500 <OptionsGod1> $ROKU Next week there will be a big storm in Shortsville  https://t.co/LUxfcOEb0I
939217431834238976 2017-12-08 13:37:40 -0500 <LalaTaylor17> Signed up fo

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
967107651145490434 2018-02-23 12:43:26 -0500 <Penniesgonewild> 👉 $NSEH 👈 Take a look at the pic below 🔥 There is somthing (Meger) in the works 😊 🚀✨  $SGMO $ROCK $CCLP $NOK $RCMH $MNKD $ZBRA $NVDA $LDSR $VEON $PEP #stocks $BGCP $ROKU  $AMAT $IFXY #OTC $RARS $ADXS $XNET $AMZN $DSGT $TSLA $QCOM #pennystocks $AABA $PZOO  https://t.co/xOvghx1Wdb
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No mo

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
1002934853980114944 2018-06-02 10:27:57 -0500 <zentrader83> Leaders setting up near support heading into the new week  $NOW $QDEL $RP $WRD $CRM $TAL $LOPE $EDU $RNG $EPAM $XCRA $MLNX $V $ERI $ZEN $SFLY $WTW $ROKU
1002747618009800704 2018-06-01 22:03:56 -0500 <kpak82> $ROKU Bull flag pullback to 10dma. Also held weekly 20sma for 2nd week in a row  https://t.co/S31rBbHn10
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
1005472566163722240 2018-06-09 10:31:55 -0500 <HiddenPivots>

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
1028424742489939968 2018-08-11 18:35:40 -0500 <OphirGottlieb> $ROKU data from @LikeFolio  https://t.co/gRCCL2cI3E
1028071845822492672 2018-08-10 19:13:23 -0500 <ioTrader> current positions: $EPAY, $TDOC, $COUP, $YEXT, $BAND, $ROKU, $WIFI, $FNKO,  5% cash
1028037199059673090 2018-08-10 16:55:42 -0500 <LSValue> I struggle with investing in anything that has gone up recently, especially something I just looked at.    My plan to address is to buy in separate blocks and some of your blocks don’t need a spectacular entry points.    Worked great with $ROKU
1028010164346077184 2018-08-10 15:08:17

...

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
1388633702108893190 2021-05-01 18:17:28 -0500 <BahamaBen9> @btyler_32 A lot of those $SQ $ROKU $TTD $TDOC $RDFN $Z $TWLO $ZS $PYPL $SHOP $DOCU $ETSY $MELI $SE $NET $FSLY $OKTA $PINS $SNAP $PTON $ZM $TSLA $CRWD
1388630886392926208 2021-05-01 18:06:17 -0500 <SJosephBurns> Earnings week:    https://t.co/eDYHYwrKxe   $PYPL $PFE $SQ $ROKU $PTON $GM $DKNG $MRNA $CVS $CRSR $RKT $SKLZ $ON $VIAC $ETSY $PENN $UBER $EL $EPD $TWLO $ATVI $TMUS $FSLY $MRO $GOLD $VSTO $SPWR $AMC $AYX $SPCE $FVRR $LPX $CHGG $NET $REGN $BYND $UAA $QRVO $CRNT $COP  via @eWhispers  https://t.co/WXgDIrSqex
138861865008139468

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
1391107319509917698 2021-05-08 14:06:45 -0500 <Trendspider_J> If you have some time this weekend, check out the weekend video, there is a ton of info into next week!  Watch here:  https://t.co/S1L8lJBsQ4  Reviewed: $SPY  $QQQ $IWM $DIA $GDX $VIX $BTCUSD  $ETHUSD #Dogecoin  $PLTR $TWTR $SQ $TSLA $ROKU  https://t.co/7aOfDKSO5R
1391070694461554689 2021-05-08 11:41:13 -0500 <HonestInvesting> Pick 10 stocks to hold for 10 years. No buying, no selling. Just holding. Who you got?  Buying at current prices.  I’ll take: $SE $SQ $TSLA $ADBE $RDFN $SHOP $NVDA $ROKU $CRWD $AMZN  All are hungry, estab

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
1393717337124184070 2021-05-15 18:58:01 -0500 <TrendSpider> $ROKU This red line seems important...  https://t.co/A0Xy7Vp2KZ
1393669487199674372 2021-05-15 15:47:53 -0500 <HarleyMCarroll> @phoenixvalue I am going with:  $SHOP, $CRWD, $NVDA, $ROKU
1393659784549838855 2021-05-15 15:09:20 -0500 <RealAniltwits> Companies that I can hold for long term in alphabetical order: What are are yours? $ADBE $ADYEY $GOOG $AMZN $APPL $CRM $CRWD $TEAM $FB $FIVN $MKTX $MA $MELI $MSFT $MRNA $MDB $NFLX $PLTR $PYPL $PAYC $PINS $NOW $ROKU $RNG $SE $SHOP $SNOW $SQ $SWAV $TSLA $TTD $TMDX $TWLO $VEEV $ZM
1393595754594381824 2021-05-15 10:54:54 -0500 <TradingWarz> 🔥Top Swing Picks for next wee

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
1398752981970001929 2021-05-29 16:27:53 -0500 <MFIndyAnthony> @WOLF_Financial 👀 Ayyyy! I'd add:  $MELI: @dannyvena  $TSLA: @JRogrow  $AAPL: @munster_gene  $RACE: @IntrinsicInv  $ROKU: @Beth_Kindig  $TTD: @dhaval_kotecha  $NVDA: @BackpackerFI  $ASML: @SeifelCapital  $TSM: @bradsling  $MTSAAS: @BessemerVP $CRM: @anandchokkavelu $OLO: @StockMarketNerd
1398741305715572741 2021-05-29 15:41:29 -0500 <FinansToppen> De 30 største posisjonene til Cathie Wood og Ark Invest hvis man legger sammen de 6 ETFene deres:  1 Tesla $TSLA $2.9B 2 Teladoc $TDOC $2B 3 Square $SQ $1.5B 4 $ROKU $1.4B 5 Shopify $

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


## Reddit Data

In [5]:
# Subreddits I want to gather data from
Reddit_subreddits = ['stocks', 'investing', 'stockmarket', 'economy', 'wallstreetbets', 'options', 'Daytrading']

### Reddit Data Test

#### Psaw/Praw

Testing Psaw/Praw package to gather reddit data

In [6]:
r = praw.Reddit(client_id=client_id,
                client_secret=client_secret,
                user_agent=user_agent)

api = PushshiftAPI(r)

In [7]:
# Setting search parameters
start_epoch=int(dt.datetime(2020, 1, 1).timestamp())
end_epoch=int(dt.datetime(2020, 2, 20).timestamp())

api_request_generator = api.search_submissions(subreddit=Reddit_subreddits,
                                               after = start_epoch, before=end_epoch,
                                               q='(ROKU)|(Roku)|(roku)|(#Roku)|(#ROKU)|(#roku)|($ROKU)|($Roku)|($roku)',
                                               limit=1000)


In [8]:
# putting data from search to a dataframe
Reddit_df = pd.DataFrame(columns =['ID','Num_Comments','Score','Subreddit','Title','Upvote_Ratio','Created',
                                   'Created_utc','Self_text'] )
ID=[]
Num_Comments=[]
Score=[]
Subreddit = []
Title = []
Upvote_Ratio = []
Created = []
Created_utc = []
Self_text = []


for submissions in api_request_generator:
    ID.append(submissions.id)
    Num_Comments.append(submissions.num_comments)
    Score.append(submissions.score)
    Subreddit.append(submissions.subreddit)
    Title.append(submissions.title)
    Upvote_Ratio.append(submissions.upvote_ratio)
    Created.append(submissions.created)
    Created_utc.append(submissions.created_utc)
    Self_text.append(submissions.selftext)
    
temp_df = pd.DataFrame({'ID':ID,
                        'Num_Comments':Num_Comments,
                        'Score':Score,
                        'Subreddit':Subreddit,
                        'Title':Title,
                        'Upvote_Ratio':Upvote_Ratio,
                        'Created':Created,
                        'Created_utc':Created_utc,
                        'Self_text':Self_text})

Reddit_df = Reddit_df.append(temp_df)
Reddit_df.sort_values(by='Score', axis=0, ascending=False, inplace=True)
Reddit_df.drop(Reddit_df[(Reddit_df['Self_text']=='[deleted]')|(Reddit_df['Self_text']=='[removed]')].index, 
              inplace=True)
Reddit_df.drop(Reddit_df[Reddit_df['Score']<21].index, inplace=True)

# Writing dataframe to a csv
dataframe_to_write = Reddit_df[0:5]
dataframe_to_write.to_csv('/Users/stevenkyle/Documents/Flatiron/Capstone/Capstone/redditdata', mode='a', header=False, index=False)

In [103]:
Reddit_df

,ID,Num_Comments,Score,Subreddit,Title,Upvote_Ratio,Created,Created_utc,Self_text
118,eq0c8i,211,1141,wallstreetbets,WSB stock mentions over the last 7 days,0.98,1.579296e+09,1.579268e+09,See you in Callhalla Brothers\n\n1 SPY - 3634 ...
65,f2idft,178,705,wallstreetbets,I crashed MSFT,0.96,1.581497e+09,1.581468e+09,I’m so sorry everyone. \n\nI watched from the ...
143,ejaqzf,117,561,wallstreetbets,"Literally the first time I buy spy calls, We g...",0.98,1.578057e+09,1.578029e+09,"I just wanna say, I'm sorry. \n\nI thought ""H..."
98,exr15m,95,346,wallstreetbets,I decided to go ahead with my Hong Kong trip t...,0.99,1.580690e+09,1.580662e+09,American refuses to give me a cash refund or c...
86,f0gt6n,15995,261,wallstreetbets,"Weekend Discussion Thread - February 7-9, 2020",0.94,1.581138e+09,1.581109e+09,Welcome back to another edition of the Weekend...
124,eo6jpw,33,226,wallstreetbets,CNBC slowly succumbing to the group they resen...,0.97,1.578962e+09,1.578934e+09,
133,ems7oz,92,201,wallstreetbets,Imagine still being bullish on Roku,0.94,1.578698e+09,1.578669e+09,Imagine making so much money on the dip last y...
13,f4rfh0,131,105,wallstreetbets,Weekly Theta Gang Thread,0.95,1.581893e+09,1.581864e+09,First week where everything didn’t go perfectl...
94,eyr5mu,14,81,wallstreetbets,WARNING - Coronavirus surpasses mentions of Au...,0.95,1.580857e+09,1.580828e+09,"7 Day chatter\n\nPosts: 1,349 \nComments: 168..."
97,ey71ww,10,73,StockMarket,"Today's Pre-Market Movers & News [Monday, Febr...",0.95,1.580765e+09,1.580736e+09,#Good morning traders and investors of the r/S...


In [100]:
Reddit_df.iloc[2]['Title']

'The Inverse WSB Algorithm - 72.5% win rate - 40 day backtest'

In [101]:
Reddit_df.iloc[2]['Self_text']

'[deleted]'

In [67]:
local_datetime_converted = dt.datetime.fromtimestamp(Reddit_df.iloc[-1]['Created_utc'])
print(local_datetime_converted)

2020-01-01 10:51:13


In [20]:
Reddit_df[0:5]

,ID,Num_Comments,Score,Subreddit,Title,Upvote_Ratio,Created,Created_utc,Self_text
118,eq0c8i,211,1137,wallstreetbets,WSB stock mentions over the last 7 days,0.98,1.579296e+09,1.579268e+09,See you in Callhalla Brothers\n\n1 SPY - 3634 ...
65,f2idft,178,705,wallstreetbets,I crashed MSFT,0.96,1.581497e+09,1.581468e+09,I’m so sorry everyone. \n\nI watched from the ...
143,ejaqzf,117,561,wallstreetbets,"Literally the first time I buy spy calls, We g...",0.98,1.578057e+09,1.578029e+09,"I just wanna say, I'm sorry. \n\nI thought ""H..."
98,exr15m,95,338,wallstreetbets,I decided to go ahead with my Hong Kong trip t...,0.99,1.580690e+09,1.580662e+09,American refuses to give me a cash refund or c...
86,f0gt6n,15995,261,wallstreetbets,"Weekend Discussion Thread - February 7-9, 2020",0.94,1.581138e+09,1.581109e+09,Welcome back to another edition of the Weekend...


In [9]:
#Possible data categories that can be gathered through psaw/praw

# Index(['all_awardings', 'allow_live_comments', 'author',
#        'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
#        'author_flair_type', 'author_fullname', 'author_patreon_flair',
#        'author_premium', 'awarders', 'can_mod_post', 'contest_mode',
#        'created_utc', 'domain', 'edited', 'full_link', 'gilded', 'gildings',
#        'id', 'is_crosspostable', 'is_meta', 'is_original_content',
#        'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video',
#        'link_flair_background_color', 'link_flair_css_class',
#        'link_flair_richtext', 'link_flair_template_id', 'link_flair_text',
#        'link_flair_text_color', 'link_flair_type', 'locked', 'media_only',
#        'no_follow', 'num_comments', 'num_crossposts', 'over_18',
#        'parent_whitelist_status', 'permalink', 'pinned', 'post_hint',
#        'preview', 'pwls', 'retrieved_on', 'score', 'selftext', 'send_replies',
#        'spoiler', 'stickied', 'subreddit', 'subreddit_id',
#        'subreddit_subscribers', 'subreddit_type', 'thumbnail', 'title',
#        'total_awards_received', 'treatment_tags', 'upvote_ratio', 'url',
#        'whitelist_status', 'wls', 'created'],
#       dtype='object')

### Looping through and gathering all Reddit Data

In [17]:
# Looping throught the dates
for i in range(len(stock_days_itter)):
    
    r = praw.Reddit(client_id=client_id,
                client_secret=client_secret,
                user_agent=user_agent)

    api = PushshiftAPI(r)
    
    # Setting parameters and doing a search
    start_epoch = stock_days_itter[i]
    
    if i < 922:
        next_day = i+1
        end_epoch = stock_days_itter[next_day]
    elif i == 922:
        end_epoch = int(dt.datetime(2021, 5, 29).timestamp())

    api_request_generator = api.search_submissions(subreddit=Reddit_subreddits,
                                               after = start_epoch, before=end_epoch,
                                               q='(ROKU)|(Roku)|(roku)|(#Roku)|(#ROKU)|(#roku)|($ROKU)|($Roku)|($roku)',
                                               limit=1000)
    
    # Adding search data to a dataframe
    Reddit_df = pd.DataFrame(columns =['ID','Num_Comments','Score','Subreddit','Title','Upvote_Ratio','Created',
                                   'Created_utc','Self_text','Date'] )
    ID=[]
    Num_Comments=[]
    Score=[]
    Subreddit = []
    Title = []
    Upvote_Ratio = []
    Created = []
    Created_utc = []
    Self_text = []
    Date = []
    

    for submissions in api_request_generator:
        ID.append(submissions.id)
        Num_Comments.append(submissions.num_comments)
        Score.append(submissions.score)
        Subreddit.append(submissions.subreddit)
        Title.append(submissions.title)
        Upvote_Ratio.append(submissions.upvote_ratio)
        Created.append(submissions.created)
        Created_utc.append(submissions.created_utc)
        Self_text.append(submissions.selftext)
        Date.append(stock_days[i])
    
    temp_df = pd.DataFrame({'ID':ID,
                            'Num_Comments':Num_Comments,
                            'Score':Score,
                            'Subreddit':Subreddit,
                             'Title':Title,
                            'Upvote_Ratio':Upvote_Ratio,
                            'Created':Created,
                            'Created_utc':Created_utc,
                            'Self_text':Self_text,
                            'Date':Date})

    Reddit_df = Reddit_df.append(temp_df)
    Reddit_df.sort_values(by='Score', axis=0, ascending=False, inplace=True)
    Reddit_df.drop(Reddit_df[(Reddit_df['Self_text']=='[deleted]')|(Reddit_df['Self_text']=='[removed]')].index, 
                  inplace=True)
    Reddit_df.drop(Reddit_df[Reddit_df['Score']<10].index, inplace=True)
    dataframe_to_write = Reddit_df[0:5]
    
    # Writing dataframe to a csv
    if i == 0:
        dataframe_to_write.to_csv(path_or_buf='/Users/stevenkyle/Documents/Flatiron/Capstone/Capstone/redditdata.csv',index=False)
    elif i > 0:
        dataframe_to_write.to_csv('/Users/stevenkyle/Documents/Flatiron/Capstone/Capstone/redditdata.csv', mode='a', header=False, index=False)
    
    time.sleep(3)

## Combining Reddit and Twitter dataframes

In [8]:
# Reading information from the csv's
R_df = pd.read_csv('redditdata.csv')
T_df = pd.read_csv('TwitterData.csv')

## Checking spacy sentiment analysis

In [2]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')
text = 'I had a really horrible day. It was the worst day ever! But every now and then I have a really good day that makes me happy.'
doc = nlp(text)
doc._.polarity      # Polarity: -0.125
doc._.subjectivity  # Sujectivity: 0.9
doc._.assessments  

[(['really', 'horrible'], -1.0, 1.0, None),
 (['worst', '!'], -1.0, 1.0, None),
 (['really', 'good'], 0.7, 0.6000000000000001, None),
 (['happy'], 0.8, 1.0, None)]